In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import torch
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report
import numpy as np
import json
from collections import defaultdict

In [3]:
SEED = 123
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
train_path = '/kaggle/input/conv-forecast/conv_derailment_data/datasets/cga/dynamic/train.json'
test_path = '/kaggle/input/conv-forecast/conv_derailment_data/datasets/cga/dynamic/test.json'
val_path = '/kaggle/input/conv-forecast/conv_derailment_data/datasets/cga/dynamic/valid.json'

train_data = []
with open(train_path,'r') as f:
  for line in f:
    train_data.append(json.loads(line))

test_data = []
with open(test_path,'r') as f:
  for line in f:
    test_data.append(json.loads(line))

val_data = []
with open(val_path,'r') as f:
  for line in f:
    val_data.append(json.loads(line))

In [6]:
# grouped_train = defaultdict(list)
# for item in train_data:
#     convo_id = item['convo_id']
#     grouped_train[convo_id].append(item)


# # duplicated convo_ids
# duplicated_convo_ids = [k for k, v in grouped_train.items() if len(v) > 1]
# print(duplicated_convo_ids)

In [7]:
train_data[0]

{'src': " i notice that earier that moved wiki _ link to bill chen citing wiki _ link, then you reverted this change, bill chen doesn't commonly go by william, his book is even penned as bill chen. from what i read in wp : commonname patrikr seems to be correct, examples given are names such as : * wiki _ link ( not wiki _ link ) * wiki _ link ( not wiki _ link ) i think this revert may have been a mistake unless you know otherwise? ▪ [UNK] ▪",
 'reply': [101,
  8802,
  2001,
  2124,
  1999,
  1996,
  11662,
  2088,
  2004,
  1000,
  2520,
  1000,
  2005,
  2086,
  2077,
  2002,
  2150,
  4141,
  2124,
  2004,
  1000,
  3021,
  1000,
  1012,
  1045,
  2904,
  2009,
  2067,
  2138,
  18949,
  2015,
  3784,
  2164,
  2224,
  7159,
  2024,
  5560,
  5020,
  1010,
  2498,
  2012,
  2035,
  2066,
  3021,
  7207,
  1998,
  2520,
  7207,
  1010,
  1998,
  1999,
  5020,
  3572,
  2478,
  1996,
  2613,
  2171,
  3849,
  1996,
  2190,
  3601,
  1012,
  1006,
  1996,
  6327,
  1035,
  4957,
  393

In [8]:
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
for item in train_data:
    item['src'] = item['src'].replace('[SEP]', '[EOS]')
    item['src'] = item['src'].replace('[UNK]', ' ')
    item['reply'] = tokenizer_bert.decode(item['reply'], skip_special_tokens=True)

for item in val_data:
    item['src'] = item['src'].replace('[SEP]', '[EOS]')
    item['src'] = item['src'].replace('[UNK]', ' ')
    item['reply'] = tokenizer_bert.decode(item['reply'], skip_special_tokens=True)

for item in test_data:
    item['src'] = item['src'].replace('[SEP]', '[EOS]')
    item['src'] = item['src'].replace('[UNK]', ' ')
    item['reply'] = tokenizer_bert.decode(item['reply'], skip_special_tokens=True)


2024-04-18 20:30:13.328027: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 20:30:13.328087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 20:30:13.329633: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
train_data[10]

{'src': ' please stop including disreputable sources for this article. wikipedia policy is quite clear on this matter blogs and other websites which do not employ editorial oversight of authors\'work are not permitted as sources here. please stop adding blogs. [EOS]  please stop deleting reputable sources from this article. you have deleted joe wilson\'s nyt article that is a key factor in this whole controversy! among others. simply because the article is printed on a different site does not mean it is sourced to a " heinous blog " ( which the site is not anyway ). if you find a better place that the article exists, put it there. or if you have trouble with the link, make a case for deleting the link, but not the whole citation! you also deleted a notable article in the notable, non - blog, journalistic source wiki _ link, a notable article from the well known and respected scholarly project wiki _ link, a notable, on - topic view of wiki _ link by wiki _ link, and a notable interview

In [11]:
def data_for_gpt2(data):
  src_text = data['src']
  reply = data['reply']
  tgt = data['tgt']
  is_last = data['is_last']
  combined_text = f"<SRC>: {src_text}  <REPLY>: {reply} <IS_LAST>: {is_last}"
  return combined_text, tgt

In [12]:
train_data_gpt2 = []
tgt_train = []
for item in train_data:
  formatted_text, tgt = data_for_gpt2(item)
  train_data_gpt2.append(formatted_text)
  tgt_train.append(tgt)

In [13]:
tgt_train[0]

False

In [14]:
train_data_gpt2[0]

'<SRC>:  i notice that earier that moved wiki _ link to bill chen citing wiki _ link, then you reverted this change, bill chen doesn\'t commonly go by william, his book is even penned as bill chen. from what i read in wp : commonname patrikr seems to be correct, examples given are names such as : * wiki _ link ( not wiki _ link ) * wiki _ link ( not wiki _ link ) i think this revert may have been a mistake unless you know otherwise? ▪   ▪  <REPLY>: chen was known in the poker world as " william " for years before he became commonly known as " bill ". i changed it back because incidences online including usenet are roughly equal, nothing at all like bill clinton and william clinton, and in equal cases using the real name seems the best choice. ( the external _ link page is especially pschizo... willam in the page title, bill in the page text ). however i suppose the book is the trump card, so using the name on the book is probably best. <IS_LAST>: False'

In [15]:
val_data_gpt2 = []
tgt_val = []
for item in val_data:
  formatted_text, tgt = data_for_gpt2(item)
  val_data_gpt2.append(formatted_text)
  tgt_val.append(tgt)

In [16]:
val_data_gpt2[0]

'<SRC>:  you have been blocked for three hours for incivility. we cannot tolerate these types of edits :  <REPLY>: on the contrary, the edits you should not tolerate are the other side preaching their incorrect disambiguation. - - wiki _ link ) <IS_LAST>: False'

In [17]:
tgt_val[0]

False

In [18]:
def test_data_for_gpt2(data):
  src_text = data['src']
  reply = data['reply']
  tgt = data['tgt']
  combined_text = f"<SRC>: {src_text}  <REPLY>: {reply} "
  return combined_text, tgt

In [19]:
test_data_gpt2 = []
tgt_test = []
for item in test_data:
  formatted_text, tgt = test_data_for_gpt2(item)
  test_data_gpt2.append(formatted_text)
  tgt_test.append(tgt)

In [20]:
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm

In [21]:
from transformers import GPT2ForSequenceClassification, GPT2Config
import torch
import torch.nn.functional as F

class GPT2ForSequenceClassificationMeanPooling(GPT2ForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.gpt2 = self.transformer  # Assuming the GPT2ForSequenceClassification model has a 'transformer' attribute
        self.score = torch.nn.Linear(config.hidden_size, config.num_labels)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,
        **kwargs
    ):
        outputs = self.gpt2(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs,
        )

        # Get the hidden states from the last layer
        hidden_states = outputs.last_hidden_state

        # Apply mean pooling on the hidden states
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
        sum_hidden_states = torch.sum(hidden_states * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        mean_hidden_states = sum_hidden_states / sum_mask

        # Pass the pooled hidden states through the classification head
        logits = self.score(mean_hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    loss_fct = torch.nn.MSELoss()
                    loss = loss_fct(logits.view(-1), labels.view(-1))
                else:
                    loss_fct = torch.nn.CrossEntropyLoss()
                    loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            else:
                raise ValueError("You should supply an instance of `PreTrainedModel` or a `config`")

        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

In [22]:
### Loading Model and Tokenizers
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',
                                         add_eos_token=True)
model = GPT2ForSequenceClassificationMeanPooling.from_pretrained('gpt2', num_labels=2)
print(model)
model.to(device)

Some weights of GPT2ForSequenceClassificationMeanPooling were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassificationMeanPooling(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=True)
  (gpt2): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop)

GPT2ForSequenceClassificationMeanPooling(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=True)
  (gpt2): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop)

In [23]:
len(tokenizer)

50257

In [24]:
tokenizer.sep_token = '[EOS]'

In [25]:
tokenizer.sep_token

'[EOS]'

In [26]:
tokenizer.pad_token = tokenizer.eos_token

In [27]:
tokenizer.pad_token

'<|endoftext|>'

In [28]:
tokenizer.eos_token

'<|endoftext|>'

In [29]:
tokenizer

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'sep_token': '[EOS]', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [30]:
tokenizer.sep_token_id

50256

In [31]:
len(tokenizer)

50257

In [32]:
## Defining Special Tokens

special_tokens = {'additional_special_tokens': ['<SRC>:', ' <REPLY>:', '[UNK]','<IS_LAST>']}
num_added = tokenizer.add_special_tokens(special_tokens)
print(num_added)
model.resize_token_embeddings(len(tokenizer))

4


Embedding(50261, 768)

In [33]:
# Tokenizing Data
train_encodings = tokenizer(train_data_gpt2, truncation=False, padding=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (1127 > 1024). Running this sequence through the model will result in indexing errors


In [34]:
tokenizer.decode(train_encodings['input_ids'][1])

'<SRC>:   i notice that earier that moved wiki _ link to bill chen citing wiki _ link, then you reverted this change, bill chen doesn\'t commonly go by william, his book is even penned as bill chen. from what i read in wp : commonname patrikr seems to be correct, examples given are names such as : * wiki _ link ( not wiki _ link ) * wiki _ link ( not wiki _ link ) i think this revert may have been a mistake unless you know otherwise? ▪   ▪ [EOS]  chen was known in the poker world as " william " for years before he became commonly known as " bill ". i changed it back because incidences online including usenet are roughly equal, nothing at all like bill clinton and william clinton, and in equal cases using the real name seems the best choice. ( the external _ link page is especially pschizo... willam in the page title, bill in the page text ). however i suppose the book is the trump card, so using the name on the book is probably best.   <REPLY>:  i see what you saying i just read his po

In [35]:
max_length = max(len(x) for x in train_encodings.input_ids)

In [36]:
max_length

1194

In [37]:
val_encodings = tokenizer(val_data_gpt2, truncation=False, padding=False)

In [38]:
val_max_length = max(len(x) for x in val_encodings.input_ids)
val_max_length

1075

In [39]:
test_encodings = tokenizer(test_data_gpt2, truncation=False, padding=False)

In [40]:
test_max_length = max(len(x) for x in test_encodings.input_ids)
test_max_length

1105

In [41]:
token_test = tokenizer(train_data_gpt2, truncation=True, padding=True, max_length=1024)

In [42]:
## Redefining Tokenizer with padding and max length
# tokenizer.pad_token = tokenizer.eos_token
train_inputs = tokenizer(train_data_gpt2, truncation=True, padding=True, max_length=1024, return_tensors='pt')

# Create torch dataset
input_ids = train_inputs['input_ids']
attention_mask = train_inputs['attention_mask']
labels = torch.tensor([0 if tgt == False else 1 for tgt in tgt_train])
train_dataset = TensorDataset(input_ids, attention_mask, labels)

In [43]:
print(input_ids[0])

tensor([50257,   220,  1312,  ..., 50256, 50256, 50256])


In [44]:
len(input_ids[0])

1024

In [45]:
val_inputs = tokenizer(val_data_gpt2, truncation=True, padding=True, max_length=1024, return_tensors='pt')
val_input_ids = val_inputs['input_ids']
val_attention_mask = val_inputs['attention_mask']
val_labels = torch.tensor([0 if tgt == False else 1 for tgt in tgt_val])
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels)

In [46]:
test_inputs = tokenizer(test_data_gpt2, truncation=True, padding=True, max_length=1024, return_tensors='pt')
test_input_ids = test_inputs['input_ids']
test_attention_mask = test_inputs['attention_mask']
test_labels = torch.tensor([0 if tgt == False else 1 for tgt in tgt_test])
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)

In [47]:
## CREATING DATALOADERS
batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [48]:
from torch.optim import AdamW

In [49]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)


In [50]:
import copy

In [51]:
from sklearn.metrics import f1_score
from tqdm import tqdm

# Training and validation loop
num_epochs = 5
train_losses = []
val_losses = []
f1_scores = []
best_val_loss = float('inf')
best_model_state = None

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1} of {num_epochs}")
    train_loss = 0
    
    model.train()
    for batch in tqdm(train_dataloader, desc='Training Progress', miniters=10):
        input_ids, attention_mask, batch_labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=batch_labels)
        loss = outputs[0]
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    avg_train_loss = train_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)
    print(f"Training loss: {avg_train_loss:.2f}")

    # Validation phase
    val_loss = 0
    model.eval()
    val_preds = []
    val_true_labels = []
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc='Validation Progress', miniters=10):
            input_ids, attention_mask, batch_labels = [b.to(device) for b in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=batch_labels)
            loss = outputs[0]
            val_loss += loss.item()
            logits = outputs[1]
            preds = torch.argmax(torch.nn.functional.softmax(logits, dim=1), dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_true_labels.extend(batch_labels.cpu().numpy())

        avg_val_loss = val_loss / len(val_dataloader)
        val_losses.append(avg_val_loss)
        f1 = f1_score(val_true_labels, val_preds, average='macro')
        f1_scores.append(f1)
        print(f"Validation loss: {avg_val_loss:.2f}, F1 score: {f1:.2f}")

    scheduler.step(avg_val_loss)

    # Check if the current model is the best one
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_state = model.state_dict()
        torch.save(best_model_state, 'CGA_Dynamic_model_GPT2.pth')  # Save the best model
        print("Saved new best model")


Epoch 1 of 5


Training Progress: 100%|██████████| 1817/1817 [17:34<00:00,  1.72it/s]


Training loss: 0.67


Validation Progress: 100%|██████████| 610/610 [01:57<00:00,  5.21it/s]


Validation loss: 0.74, F1 score: 0.55
Saved new best model
Epoch 2 of 5


...

Validation loss: 1.27, F1 score: 0.60
Epoch 5 of 5


Training Progress: 100%|██████████| 1817/1817 [17:33<00:00,  1.72it/s]


Training loss: 0.05


Validation Progress: 100%|██████████| 610/610 [01:57<00:00,  5.21it/s]

Validation loss: 1.36, F1 score: 0.60
Epoch 00005: reducing learning rate of group 0 to 1.0000e-07.


In [52]:
model.eval()
test_loss = 0
test_preds = []
test_true_labels = []
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc='Test Progress', miniters = 10):
        input_ids, attention_mask, batch_labels = batch
        input_ids, attention_mask, batch_labels = input_ids.to(device), attention_mask.to(device), batch_labels.to(device)
        model.config.pad_token_id = tokenizer.eos_token_id
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=batch_labels)
        loss = outputs[0]  # Extract the loss value from the tuple
        test_loss += loss.item()
        logits = outputs[1] 
        probs = torch.nn.functional.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)
        test_preds.extend(preds.cpu().numpy())
        test_true_labels.extend(batch_labels.cpu().numpy())
        
avg_test_loss = test_loss / len(test_dataloader)
f1 = f1_score(test_true_labels, test_preds, average='macro')
print(f"Test loss: {avg_test_loss:.2f}, F1 score: {f1:.2f}")

Test Progress: 100%|██████████| 1069/1069 [03:25<00:00,  5.21it/s]

Test loss: 1.43, F1 score: 0.58


In [54]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

F1_score = f1_score(test_true_labels, test_preds, average='macro')
accuracy = accuracy_score(test_true_labels, test_preds)
precision = precision_score(test_true_labels, test_preds, average='macro')
recall = recall_score(test_true_labels, test_preds, average='macro')

print(f"F1 score: {F1_score}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

F1 score: 0.5788503119664257
Accuracy: 0.584093567251462
Precision: 0.5836265389610066
Recall: 0.5808115452891801


In [55]:
conf_matrix = confusion_matrix(test_true_labels, test_preds)
classification_report = classification_report(test_true_labels, test_preds)

print(conf_matrix)
print(classification_report)

[[1010 1050]
 [ 728 1487]]
              precision    recall  f1-score   support

           0       0.58      0.49      0.53      2060
           1       0.59      0.67      0.63      2215

    accuracy                           0.58      4275
   macro avg       0.58      0.58      0.58      4275
weighted avg       0.58      0.58      0.58      4275



In [56]:
data = {'True Labels': test_true_labels, 'Predictions': test_preds}
prediction_comparison_df = pd.DataFrame(data)

In [59]:
prediction_comparison_df

,True Labels,Predictions
0,0,1
1,0,1
2,1,1
3,1,1
4,0,0
...,...,...
4270,0,1
4271,0,1
4272,0,1
4273,0,1


# LOADING BEST SAVED MODEL 

In [60]:
model_1 = GPT2ForSequenceClassificationMeanPooling.from_pretrained('gpt2', num_labels=2)
model_1.resize_token_embeddings(len(tokenizer)) 
model_1.load_state_dict(torch.load('/kaggle/working/CGA_Dynamic_model_GPT2.pth', map_location=device))
model_1.to(device)

Some weights of GPT2ForSequenceClassificationMeanPooling were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassificationMeanPooling(
  (transformer): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=True)
  (gpt2): GPT2Model(
    (wte): Embedding(50261, 768)
    (wpe): Embedding(1024, 768)
    (drop)

In [61]:
model_1.eval()
test_loss = 0
test_preds = []
test_true_labels = []
with torch.no_grad():
    for batch in tqdm(test_dataloader, desc='Test Progress', miniters = 10):
        input_ids, attention_mask, batch_labels = batch
        input_ids, attention_mask, batch_labels = input_ids.to(device), attention_mask.to(device), batch_labels.to(device)
        model_1.config.pad_token_id = tokenizer.eos_token_id
        outputs = model_1(input_ids=input_ids, attention_mask=attention_mask, labels=batch_labels)
        loss = outputs[0]  # Extract the loss value from the tuple
        test_loss += loss.item()
        logits = outputs[1] 
        probs = torch.nn.functional.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)
        test_preds.extend(preds.cpu().numpy())
        test_true_labels.extend(batch_labels.cpu().numpy())
        
avg_test_loss = test_loss / len(test_dataloader)
f1 = f1_score(test_true_labels, test_preds, average='macro')
print(f"Test loss: {avg_test_loss:.2f}, F1 score: {f1:.2f}")

Test Progress: 100%|██████████| 1069/1069 [03:25<00:00,  5.21it/s]

Test loss: 0.71, F1 score: 0.56
